In [67]:
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import Counter
import glob
from nltk.corpus import stopwords


In [68]:
#reading files 
def load_data_from_folder(folder_path, label):
    emails = []
    for filepath in glob.glob(os.path.join(folder_path, '*')):
        try:
            with open(filepath, 'r', encoding='latin-1') as f:
                content = f.read()
                emails.append({'text': content, 'label': label})
        except Exception as e:
            print(f"Error reading {filepath}: {e}")
    return emails

In [69]:
ham = load_data_from_folder(r"C:\Users\asus\OneDrive\Documents\SIT Notes 2025\PRG Fundamentals\Python Project\Inbox-Gaurdian\ML Data\easy_ham\easy_ham", "ham")
spam = load_data_from_folder(r"C:\Users\asus\OneDrive\Documents\SIT Notes 2025\PRG Fundamentals\Python Project\Inbox-Gaurdian\ML Data\spam_2\spam_2", "spam")
overall = ham + spam

In [70]:
data = pd.DataFrame(overall)
print(data.head())
print(data['label'].value_counts())

                                                text label
0  From exmh-workers-admin@redhat.com  Thu Aug 22...   ham
1  From Steve_Burt@cursor-system.com  Thu Aug 22 ...   ham
2  From timc@2ubh.com  Thu Aug 22 13:52:59 2002\n...   ham
3  From irregulars-admin@tb.tf  Thu Aug 22 14:23:...   ham
4  From exmh-users-admin@redhat.com  Thu Aug 22 1...   ham
label
ham     2551
spam    1397
Name: count, dtype: int64


In [71]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def clean_email_text(text):
        # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    # Remove CSS-style terms
    text = re.sub(r'(font|style|color|size|mso|align|margin|padding|width|height|serif|sans|helvetica|tahoma)', ' ', text, flags=re.IGNORECASE)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', ' ', text)
    # Remove numbers and special chars
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    return text

In [73]:
stop_words = set(stopwords.words("english"))
markup_words = {
    "font", "style", "color", "size", "mso", "align", "margin", "padding",
    "width", "height", "serif", "sans", "helvetica", "tahoma", "div", "span", "table", "html", "body"
}

def clean_email(text):
    # Remove headers (optional)
    text = re.sub(r'(?s)^.*?\n\n', '', text)
    # Use your HTML/CSS cleaning function
    text = clean_email_text(text)
    # Lowercase + remove non-letter chars
    text = re.sub(r'[^a-zA-Z\s]', ' ', text.lower())
    # Remove stopwords and markup words
    tokens = [word for word in text.split() if word not in stop_words and word not in markup_words]
    return ' '.join(tokens)

data["cleaned_text"] = data["text"].apply(clean_email)

In [74]:
from collections import Counter

phish_words = Counter()
ham_words = Counter()

for _, row in data.iterrows():
    words = row["cleaned_text"].split()
    if row["label"] == "spam":
        phish_words.update(words)
    else:
        ham_words.update(words)

In [75]:
import math
suspicion_scores = {}

for word in phish_words:
    phish_count = phish_words[word]
    ham_count = ham_words.get(word, 0)
    score = (phish_count / (ham_count + 1)) * math.log(phish_count + 1)
    suspicion_scores[word] = round(score, 3)

In [76]:
filtered_scores = {
    w: s for w, s in suspicion_scores.items()
    if phish_words[w] > 5 and s > 1.5  # adjust thresholds
}

In [ ]:
import json

with open("scam_dictionary.json", "w") as f:
    json.dump(filtered_scores, f, indent=4)

# Remove markup words from filtered_scores
final_scores = {w: s for w, s in filtered_scores.items() if w not in markup_words}

sorted_dict = dict(sorted(final_scores.items(), key=lambda x: x[1], reverse=True))

for word, score in sorted_dict.items():
    print(f"{word}: {score}")

mv: 5166.362
darial: 2023.866
bd: 1812.679
fx: 1765.869
dqo: 1738.975
dd: 1066.962
toner: 1029.232
cbr: 1004.182
fb: 988.139
dagga: 923.355
ragga: 904.835
bf: 902.667
cartridge: 880.222
nbsp: 828.482
deathtospamdeathtospamdeathtospam: 825.188
eb: 822.632
yni: 806.953
zm: 806.953
bc: 770.791
ctd: 770.654
ftd: 770.654
tablets: 752.592
cdovl: 746.585
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa: 728.603
zvbnq: 722.623
botanical: 716.65
zt: 710.683
ae: 608.857
decoration: 604.535
awr: 592.895
udd: 592.895
enenkio: 581.287
ba: 576.177
fd: 559.552
fa: 555.723
ic: 548.421
amp: 543.977
cj: 523.753
xvcj: 506.666
capillaris: 489.661
herba: 489.661
ncg: 484.012
refinance: 467.122
verdana: 467.122
lkdgg: 461.512
vd: 461.512
aeopublishing: 450.322
mortgage: 440.945
intro: 439.172
ef: 437.041
dha: 437.041
iahr: 416.999
laserjet: 411.483
fstrong: 411.483
fe: 406.54
ftr: 400.483
imh: 389.528
cstrong: 378.62
kdqo: 373.183
ce: 365.476
cdiv: 362.345
fdiv: 362.345
ctr: 356.

: 